# Lab 05 - Word Embeddings
In this lab we will look into word embeddings with word2vec and other similar methods

### Build your own
Let's start by first building out own word2vec model, instead of downloading a ready trained one. For that we are going to use the 20 news groups from sklearn, since is not too large for a lab exercise.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

# lets check the first two documents
documents[:2]

The first thing to do is to format the documents into a list of sentences that contains a list of tokens. We are not going to do any further cleaning and pre-processing for now (to keep things simple for the labs), but that would be advisable.

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

# This will take a minute or so
token_list = []
for d in documents:
    s = sent_tokenize(d) 
    token_list = token_list +[word_tokenize(t) for t in s]

# check the first three sentences
token_list[:3]

Now is time to import the word2vec algorithm and set the key parameters

In [ ]:
from gensim.models.word2vec import Word2Vec

# Number of vector elements (dimensions) to represent the word vector
num_features = 300
# Min number of word count to be considered in the Word2vec model. If your corpus is small, reduce the min count. If you’re training with a large corpus, increase the min count.
min_word_count = 1
# Number of CPU cores used for the training. If you want to set the number of cores dynamically, check out import multiprocessing: 
#num_workers = multiprocessing.cpu_count()
num_workers = 2
# Context window size
window_size = 3
# Subsampling rate for frequent terms
subsampling = 1e-3

Let's train the model!

In [ ]:
%%time
model = Word2Vec(token_list, workers=num_workers, vector_size=num_features, min_count=min_word_count, window=window_size, sample=subsampling)

Once you’ve trained your word model, you can reduce the memory footprint by about half if you freeze your model and discard the unnecessary information. The following command will discard the unneeded output weights of your neural network:

The model cannot be trained further once the weights of the output layer have been discarded.

Save the trained model with the following command and preserve it for later use:

In [ ]:
model_name = "my_own_domain_specific_word2vec_model"
model.save(model_name)

Now lets say we want to load the model that we had previously saved.

In [ ]:
from gensim.models.word2vec import Word2Vec
model_name = "my_own_domain_specific_word2vec_model"
model = Word2Vec.load(model_name)

Let's check the most similar words to "justice"

In [ ]:
print(model.wv.most_similar('justice'))

### Challenge - 1
Try a few more words and observe if what is retrieved makes sense

### Using the gensim API
Having build our own model is great, but lets now load a model that was trained with MANY documents.

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

We will be using the downloader for the embedding models

In [ ]:
import gensim.downloader as api

# this command can be used to check what models are available
#api.info()

Let's load the word2vec model from google news containing 300 features

In [ ]:
# This will also take a minute or so
word2vec_model = api.load('word2vec-google-news-300')

Now check the embedding vector for "beautiful"... you will see a 300 dimensional vector

In [ ]:
word2vec_model["beautiful"]

Let's check some similar words to the word "girl"

In [ ]:
word2vec_model.most_similar("girl")

How about some maths with vectors! Try the following:

queen - girl + boy = king

In [ ]:
word2vec_model.most_similar(positive=['boy', 'queen'], negative=['girl'], topn=1)

Time to do some visualisations and see how similar words end up close together and far from other words that are not as similar

In [ ]:
vocab = ["boy", "girl", "man", "woman", "king", "queen", "banana", "apple", "mango", "fruit", "coconut", "orange"]

def tsne_plot(model):
    labels = []
    wordvecs = []

    for word in vocab:
        wordvecs.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=3, n_components=2, init='pca', random_state=42)
    coordinates = tsne_model.fit_transform(wordvecs)

    x = []
    y = []
    for value in coordinates:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(8,8)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(2, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

tsne_plot(word2vec_model)

### Challenge - 2
Try a few more examples to visualise and see if similar words land close together

## GloVe
Let's try another model (GloVe) and see if that is any different to word2vec

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
import gensim.downloader as api
glove_model = api.load('glove-wiki-gigaword-300')

In [ ]:
glove_model["beautiful"]

It will be interesting to see if this will fins similar words to "girl" like word2vec did

In [ ]:
glove_model.most_similar("girl")

Let's also see if it can solve the same analogy too

In [ ]:
glove_model.most_similar(positive=['boy', 'queen'], negative=['girl'], topn=1)

In [ ]:
vocab = ["boy", "girl", "man", "woman", "king", "queen", "banana", "apple", "mango", "fruit", "coconut", "orange"]

def tsne_plot(model):
    labels = []
    wordvecs = []

    for word in vocab:
        wordvecs.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=3, n_components=2, init='pca', random_state=42)
    coordinates = tsne_model.fit_transform(wordvecs)

    x = []
    y = []
    for value in coordinates:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(8,8)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(2, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

tsne_plot(glove_model)

Let's continue with GloVe and check if plural words play any role in how close is to the original singular words

In [ ]:
print(glove_model.distance("fruit", "fruits"))
print(glove_model.distance("girl", "girls"))
print(glove_model.distance("girl", "boy"))

## Challenge - 3
Calculate the distance for "king" and "queen", then for "woman and "man". Is it similar? Check the plot to confirm.

Calculate the distance for "king" and "apple", then for "queen" and "apple". Is it similar again? Check the plot to confirm.

Now let's try and see if the model can find the capitals of different countries

In [ ]:
import pandas as pd
# pretty print function
def pp(obj):
    print(pd.DataFrame(obj))
    
def analogy(worda, wordb, wordc):
    result = glove_model.most_similar(negative=[worda], positive=[wordb, wordc], topn=1)
    return result[0][0]

countries = ['australia', 'canada', 'germany', 'ireland', 'italy']
capitals = [analogy('usa', 'washington', country) for country in countries]
pp(zip(countries,capitals))

## Challenge - 4
Looks good... but what if you change "usa" to "us"? Or if you used a different example to start with like "greece" and "athens"?

Now let's plot the results on a graph

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

def plot_data(orig_data, labels):
    pca = PCA(n_components=2)
    data = pca.fit_transform(orig_data)
    plt.figure(figsize=(7, 5), dpi=100)
    plt.plot(data[:,0], data[:,1], '.')
    for i in range(len(data)):
        plt.annotate(labels[i], xy = data[i])
    for i in range(len(data)//2):
        plt.annotate("",
                xy=data[i],
                xytext=data[i+len(data)//2],
                arrowprops=dict(arrowstyle="->",
                                connectionstyle="arc3")
        )
       
labels = countries + capitals
data = [glove_model[w] for w in labels]
plot_data(data, labels)

## doc2vec
Now let's look into generating feature vectors for documents instead of just words. For that we are going to use word2vec

In [ ]:
import multiprocessing
num_cores = multiprocessing.cpu_count()

from gensim.models.doc2vec import TaggedDocument,Doc2Vec
from gensim.utils import simple_preprocess

First let's load some data

In [ ]:
corpus = dataset.data

training_corpus = []
for i, text in enumerate(corpus):
    tagged_doc = TaggedDocument(simple_preprocess(text), [i])
    training_corpus.append(tagged_doc)
    
# If you’re running low on RAM, and you know the number of documents ahead of time (your corpus object isn’t an iterator or generator),
# you might want to use a preallocated numpy array instead of Python list for your training_corpus:
#training_corpus = np.empty(len(corpus), dtype=object);
#… 
#training_corpus[i] = …

Now we will build the model and train it

In [ ]:
doc2vec_model = Doc2Vec(vector_size=100, min_count=2, workers=num_cores, epochs=10)
doc2vec_model.build_vocab(training_corpus)
doc2vec_model.train(training_corpus, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

In [ ]:
doc2vec_model_2 = Doc2Vec(vector_size=300, min_count=2, workers=num_cores, epochs=10)
doc2vec_model_2.build_vocab(training_corpus)
doc2vec_model_2.train(training_corpus, total_examples=doc2vec_model_2.corpus_count, epochs=doc2vec_model_2.epochs)

In [ ]:
doc2vec_model_3 = Doc2Vec(vector_size=100, min_count=2, workers=num_cores, epochs=20)
doc2vec_model_3.build_vocab(training_corpus)
doc2vec_model_3.train(training_corpus, total_examples=doc2vec_model_3.corpus_count, epochs=doc2vec_model_3.epochs)

In [ ]:
doc2vec_model_4 = Doc2Vec(vector_size=300, min_count=2, workers=num_cores, epochs=20)
doc2vec_model_4.build_vocab(training_corpus)
doc2vec_model_4.train(training_corpus, total_examples=doc2vec_model_4.corpus_count, epochs=doc2vec_model_4.epochs)

Time to generate the feature vector of a new document!

In [ ]:
doc2vec_model.infer_vector(simple_preprocess('This is a completely unseen document'), epochs=10)
doc_vector

In [ ]:
doc2vec_model_2.infer_vector(simple_preprocess('This is a completely unseen document'), epochs=10)
doc_vector

In [ ]:
doc2vec_model_3.infer_vector(simple_preprocess('This is a completely unseen document'), epochs=20) # Use the increased number of epochs
doc_vector

In [ ]:
doc2vec_model_4.infer_vector(simple_preprocess('This is a completely unseen document'), epochs=20) # Use the increased number of epochs
doc_vector

Time to generate the feature vector of a new document! In this instance I just chose an existing one (i.e. the first one).

In [ ]:
doc_vector = doc2vec_model.infer_vector(simple_preprocess(dataset.data[0]), epochs=10)
doc_vector

I will now try to find the most similar documents. This will return the index of the most similar docs. Since I used an existing document as in input, I am expecting this (i.e. index 0) to be the best match.

In [ ]:
doc2vec_model.docvecs.most_similar([doc_vector])

Now lets check the two most similar documents and see if there is similarity in the context.

In [ ]:
print(" *** Doc 1 ***")
print(dataset.data[0])
print(" *** Doc 2 ***")
print(dataset.data[3580])

## Challenge - 5
Use the fetch_20newsgroups dataset from sklearn (see code above) and re-train doc2vec with that data instead.

Then, check using the most similar function to see if the documents you test are indeed similar.